In [1]:
!pip install transformers
!pip install nltk
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import files

uploaded = files.upload()


Saving cleaned_reviews.csv to cleaned_reviews.csv


In [3]:
import pandas as pd

df = pd.read_csv("cleaned_reviews.csv")


In [4]:
from transformers import pipeline

# Load model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Apply on reviews
df['sentiment_result'] = df['review'].astype(str).apply(lambda x: sentiment_pipeline(x[:512])[0])  # truncate if needed
df['sentiment_label'] = df['sentiment_result'].apply(lambda x: x['label'])
df['sentiment_score'] = df['sentiment_result'].apply(lambda x: x['score'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [5]:
import spacy
import re
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))  # remove punctuation/numbers
    doc = nlp(text.lower())
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df['clean_review'] = df['review'].apply(preprocess)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=100, ngram_range=(1,2))  # unigrams and bigrams
X = tfidf.fit_transform(df['clean_review'])

keywords = tfidf.get_feature_names_out()
print(keywords[:20])  # example


['access' 'account' 'add' 'allow' 'amazing' 'amole' 'android' 'app'
 'app not' 'application' 'ask' 'bad' 'bad app' 'bank' 'banking'
 'banking app' 'boa' 'bug' 'cbe' 'crash']


In [7]:
def assign_theme(text):
    themes = []
    if any(k in text for k in ['login', 'password', 'access']):
        themes.append('Login Issues')
    if any(k in text for k in ['slow', 'crash', 'freeze']):
        themes.append('Performance')
    if any(k in text for k in ['interface', 'design', 'layout']):
        themes.append('UI/UX')
    if any(k in text for k in ['help', 'support', 'contact']):
        themes.append('Customer Support')
    if any(k in text for k in ['fingerprint', 'feature', 'add']):
        themes.append('Feature Request')
    return ', '.join(themes) if themes else 'Other'

df['themes'] = df['clean_review'].apply(assign_theme)


In [8]:
df.to_csv("bank_reviews_sentiment_themes.csv", index=False)
